# Skin cancer

Skin cancers are cancers that arise from the skin. They are due to the development of abnormal cells that have the ability to invade or spread to other parts of the body.



![](https://media.springernature.com/lw685/springer-static/image/art%3A10.1007%2Fs11042-020-09388-2/MediaObjects/11042_2020_9388_Fig3_HTML.png)


## Dataset Link 
[Here](https://www.kaggle.com/c/siim-isic-melanoma-classification/data)

## What is VGG16 model?
VGG16 (also called OxfordNet) is a convolutional neural network architecture named after the Visual Geometry Group from Oxford, who developed it. ... By only keeping the convolutional modules, our model can be adapted to arbitrary input sizes. The model loads a set of weights pre-trained on ImageNet.

![](https://neurohive.io/wp-content/uploads/2018/11/vgg16.png)

In [ ]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from tensorflow.keras.applications.resnet50 import ResNet50 
#,NASNetLarge
from keras.layers import Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense,Dropout
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
import warnings
warnings.filterwarnings("ignore")

#from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
from tensorflow.keras.optimizers import Adam ,RMSprop
import matplotlib.pyplot as plt
from tensorflow.keras.applications.vgg16 import VGG16

In [ ]:
image_width, image_height = 224,224

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=20,
      shear_range=0.2,
      zoom_range=0.2,
      width_shift_range=0.3,
      height_shift_range=0.3,
      horizontal_flip=True,
      fill_mode='nearest')

test_datagen = ImageDataGenerator(rescale = 1./255.)

train_generator = train_datagen.flow_from_directory(
                    "../input/datasetcan/train",
                    batch_size=128,
                    class_mode='categorical',
                    shuffle=True,
                    target_size=(image_width, image_height)
)     

test_generator =  test_datagen.flow_from_directory(
                    "../input/datasetcan/test",
                    batch_size=128,
                    class_mode='categorical',
                    shuffle=True,
                    target_size=(image_width, image_height)
)

In [ ]:
train_steps = train_generator.n // train_generator.batch_size

print(train_steps)

In [ ]:
test_steps = test_generator.n // test_generator.batch_size

print(test_steps)

In [ ]:
vvg = VGG16(input_shape=(224,224, 3), include_top=False,weights = 'imagenet')

In [ ]:
for layers in (vvg.layers):
    layers.trainable = False

In [ ]:
vvg.layers

In [ ]:
model = Sequential()

# Add the ResNet50 convolutional base model
model.add(vvg)

# Add new layers
model.add(Flatten())
model.add(Dense(4096 , activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(4096 , activation='relu'))
model.add(Dropout(0.3))
model.add(Dense(4096, activation='relu'))
model.add(Dropout(0.4))
model.add(Dense(2, activation='softmax'))

model.summary()

In [ ]:
from tensorflow.keras.utils import plot_model
from IPython.display import Image
plot_model(model, to_file='convnet.png', show_shapes=True,show_layer_names=True)
Image(filename='convnet.png') 

In [ ]:
lr = 1e-6
model.compile(loss='categorical_crossentropy', optimizer=Adam(lr), metrics=['accuracy'])

In [ ]:
mcp = ModelCheckpoint('modelVVG.h5', verbose=1)

In [ ]:
es = EarlyStopping(patience=2,verbose=1)

In [ ]:
history = model.fit(train_generator,steps_per_epoch=train_steps,epochs=30,validation_data=test_generator,validation_steps=test_steps,verbose=1,callbacks=[mcp,es])

In [ ]:
model.evaluate(test_generator, verbose=1, steps=test_steps)

In [ ]:
import matplotlib.pyplot as plt

accuracy      = history.history['accuracy']
val_accuracy  = history.history['val_accuracy']
loss     = history.history['loss']
val_loss = history.history['val_loss']

epochs   = range(len(accuracy))

plt.plot(epochs, accuracy)
plt.plot(epochs, val_accuracy)
plt.title('Training and validation accuracy')
plt.figure()

# Plot training and validation loss per epoch
plt.plot(epochs, loss)
plt.plot(epochs, val_loss)
plt.title('Training and validation loss')

In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model

image_width, image_height = 224,224

img = image.load_img('../input/siim-isic-melanoma-classification/jpeg/test/ISIC_0052060.jpg', target_size=(image_width, image_height))
img = image.img_to_array(img)
img = np.expand_dims(img, axis = 0)
img/= 255.
model = load_model('modelVVG.h5')
result = model.predict(img)
train_generator.class_indices
if result[0][0] == 0:
    prediction = 'Malignant'
else:
    prediction = 'Benign'
print(prediction)